update кода для обработки fasta файла

https://colab.research.google.com/drive/1zD-0i3rjr_5m1d4zqMVXGN0WGxz9g-UJ?hl=ru#scrollTo=tfk-1Yn3LXsw


In [1]:
class Seq:
    """
    класс для представления биологической последовательности

    sequence  нуклеотидная или аминокислотная последовательность
    header заголовок FASTA записи (информация о последовательности)
    """

    def __init__(self, sequence, header=""):

        self.sequence = sequence
        self.header = header

    def __str__(self):

        return f">{self.header}\n{self.sequence}" #строковое представление последовательности

    def __len__(self):

        return len(self.sequence)

    def alphabet(self):
        """
        определяет тип последовательности нукл или протеин

        """
        nucleotides = set("ATGCU")
        amino_acids = set("ACDEFGHIKLMNPQRSTVWY")

        seq_set = set(self.sequence)

        if seq_set.issubset(nucleotides): #проверка что одно множество может быть подмножеством другого
            return "nucleotide"
        elif seq_set.issubset(amino_acids):
            return "protein"
        else:
            return "unknown"


class FastaReader:
    """
    класс для чтения FASTA файлов
    """

    def is_fasta(self, rna):
        """
        проверяет является ли файл FASTA файлом анализируя первую строку

        """
        with open(rna, 'r') as f:
            first_line = f.readline().strip()
            return first_line.startswith(">")

    def read_records(self, rna):
        """
        читает FASTA файл по записям
        + генератор чтобы не случился stack over flow

        """


        header = None
        sequence = ""

        with open(rna, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith(">"): # идентификатор фасты
                    if header: #если это не первая запись
                        yield Seq(sequence, header) # seq генератор, представляющий одну FASTA запись
                    header = line[1:] # удалям символ > в начале
                    sequence = "" # сброс
                else:
                    sequence += line

            if header:
                yield Seq(sequence, header)

    def analyze_fasta(self, rna):
        """
        анализирует FASTA файл и возвращает статистику

        возвращает словарь с:
        - total_sequences: общее количество последовательностей
        - average_length: средняя длина последовательности
        - records: генератор записей (для обработки в цикле)
        """
        total_sequences = 0
        total_length = 0
        records = []

        # собираем все записи для анализа
        for record in self.read_records(rna):
            records.append(record)
            total_sequences += 1
            total_length += len(record)

        # вычисляем среднюю длину
        average_length = total_length / total_sequences if total_sequences > 0 else 0

        # возвращаем статистику и генератор для записей
        return {
            'total_sequences': total_sequences,
            'average_length': average_length,
            'records': (record for record in records)  # возвращаем генератор
        }


if __name__ == "__main__":
    rna = "rna.fna"
    reader = FastaReader()


    analysis = reader.analyze_fasta(rna)

    print(f"Общее количество последовательностей: {analysis['total_sequences']}")
    print(f"Средняя длина последовательности: {analysis['average_length']:.2f}")



    for record in analysis['records']:
        print(f"Header: {record.header}")
        print(f"Length: {len(record)}")
        print(f"Bio type: {record.alphabet()}")


Общее количество последовательностей: 3
Средняя длина последовательности: 3714.33
Header: NM_001197320.2 MCL1 [organism=Homo sapiens] [GeneID=4170] [transcript=3]
Length: 3491
Bio type: nucleotide
Header: NM_021960.5 MCL1 [organism=Homo sapiens] [GeneID=4170] [transcript=1]
Length: 3950
Bio type: nucleotide
Header: NM_182763.3 MCL1 [organism=Homo sapiens] [GeneID=4170] [transcript=2]
Length: 3702
Bio type: nucleotide
